In [1]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
"""
Output size = (Input size - Kernel size + 2 × Padding) / Stride + 1

# Parameters:
Input size = 200
Kernel size = 3
Padding = 0 (default)
Stride = 1 (default)

# Calculation:
Output size = (200 - 3 + 2 × 0) / 1 + 1
           = (200 - 3 + 0) / 1 + 1
           = 197 / 1 + 1
           = 197 + 1
           = 198
"""

In [2]:
import torch.nn as nn

class HairClassifier(nn.Module):
    def __init__(self):
        super(HairClassifier, self).__init__()
        
        # 1. Convolutional layer: 32 filters, 3x3 kernel
        self.conv1 = nn.Conv2d(
            in_channels=3,      # RGB input
            out_channels=32,    # 32 filters
            kernel_size=3       # 3x3 kernel
        )
        self.relu1 = nn.ReLU()
        
        # 2. Max pooling layer: 2x2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # 3. Flatten (implemented in forward method)
        
        # 4. Fully connected layer: 64 neurons
        # Calculate input size:
        # Input: (3, 200, 200)
        # After Conv2d(3x3): (32, 198, 198)
        # After MaxPool2d(2x2): (32, 99, 99)
        # Flatten: 32 * 99 * 99 = 313,632
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.relu2 = nn.ReLU()
        
        # 5. Output layer: 1 neuron (binary classification)
        self.fc2 = nn.Linear(64, 1)
        # Note: No sigmoid here because we use BCEWithLogitsLoss
    
    def forward(self, x):
        # Conv + ReLU
        x = self.conv1(x)
        x = self.relu1(x)
        
        # MaxPooling
        x = self.pool(x)
        
        # Flatten
        x = x.view(x.size(0), -1)  # or use torch.flatten(x, 1)
        
        # FC + ReLU
        x = self.fc1(x)
        x = self.relu2(x)
        
        # Output
        x = self.fc2(x)
        
        return x

# Create model
model = HairClassifier()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

HairClassifier(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (relu2): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

## Question 1

Answer: nn.BCEWithLogitsLoss(), because BCEWithLogitsLoss = Sigmoid + Binary Cross Entropy

* nn.MSELoss() - Used for regression tasks
* nn.CrossEntropyLoss() - Used for multi-class classification (requires multiple output neurons)
* nn.CosineEmbeddingLoss() - Used for similarity learning

## Question 2

In [3]:
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------
Total parameters: 20073473


Answer: 20073473

### Generators and Training

In [5]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = datasets.ImageFolder(
    root='./data/train',
    transform=train_transforms
)

test_dataset = datasets.ImageFolder(
    root='./data/test',
    transform=train_transforms
)

train_loader = DataLoader(
    train_dataset,
    batch_size=20,
    shuffle=True  
)

test_loader = DataLoader(
    test_dataset,
    batch_size=20,
    shuffle=False  
)

In [7]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()

num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) 
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)
    
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)
    
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6665, Acc: 0.6112, Val Loss: 0.6511, Val Acc: 0.6617
Epoch 2/10, Loss: 0.5702, Acc: 0.6787, Val Loss: 0.6332, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5207, Acc: 0.7350, Val Loss: 0.6143, Val Acc: 0.6766
Epoch 4/10, Loss: 0.4773, Acc: 0.7600, Val Loss: 0.6049, Val Acc: 0.6617
Epoch 5/10, Loss: 0.4606, Acc: 0.7550, Val Loss: 0.7307, Val Acc: 0.5672
Epoch 6/10, Loss: 0.3954, Acc: 0.8275, Val Loss: 0.6412, Val Acc: 0.6866
Epoch 7/10, Loss: 0.2844, Acc: 0.8838, Val Loss: 0.8307, Val Acc: 0.6816
Epoch 8/10, Loss: 0.2885, Acc: 0.8788, Val Loss: 0.7052, Val Acc: 0.7114
Epoch 9/10, Loss: 0.1882, Acc: 0.9313, Val Loss: 0.9275, Val Acc: 0.6866
Epoch 10/10, Loss: 0.2585, Acc: 0.8912, Val Loss: 0.8158, Val Acc: 0.6915


## Question 3

In [8]:
import numpy as np

median_train_acc = np.median(history['acc'])
print(f"Median training accuracy: {median_train_acc:.2f}")

Median training accuracy: 0.79


Answer: 0.84, closest one.

## Question 4

In [9]:
std_train_loss = np.std(history['loss'])
print(f"Std of training loss: {std_train_loss:.3f}")

Std of training loss: 0.146


Answer: 0.171, the closest one.

### Data Augmentation

In [10]:
train_transforms_aug = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),  
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)), 
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset_aug = datasets.ImageFolder(
    root='./data/train',
    transform=train_transforms_aug
)

train_loader_aug = DataLoader(
    train_dataset_aug,
    batch_size=20,
    shuffle=True
)

In [11]:
# Continue training for 10 more epochs (Epoch 11-20) with augmentation
num_epochs = 10  # Train for 10 more epochs

for epoch in range(num_epochs):  # This will be epochs 0-9 in this loop
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader_aug:  # Use augmented data loader
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_dataset_aug)  # Use augmented dataset length
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)
    
    # Validation phase
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)
    
    # Print with correct epoch number (11-20)
    actual_epoch = len(history['loss'])  # Total epochs so far
    print(f"Epoch {actual_epoch}/20, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 11/20, Loss: 0.6472, Acc: 0.6538, Val Loss: 0.6916, Val Acc: 0.7114
Epoch 12/20, Loss: 0.6190, Acc: 0.6475, Val Loss: 0.6385, Val Acc: 0.6368
Epoch 13/20, Loss: 0.5847, Acc: 0.6837, Val Loss: 0.6113, Val Acc: 0.6667
Epoch 14/20, Loss: 0.5550, Acc: 0.7137, Val Loss: 0.5822, Val Acc: 0.6965
Epoch 15/20, Loss: 0.5496, Acc: 0.7125, Val Loss: 0.5541, Val Acc: 0.6915
Epoch 16/20, Loss: 0.5193, Acc: 0.7212, Val Loss: 0.6146, Val Acc: 0.6816
Epoch 17/20, Loss: 0.5227, Acc: 0.7312, Val Loss: 0.5665, Val Acc: 0.6965
Epoch 18/20, Loss: 0.5223, Acc: 0.7175, Val Loss: 0.6257, Val Acc: 0.6617
Epoch 19/20, Loss: 0.5166, Acc: 0.7412, Val Loss: 0.5468, Val Acc: 0.7114
Epoch 20/20, Loss: 0.5142, Acc: 0.7462, Val Loss: 0.5622, Val Acc: 0.7214


## Question 5

In [12]:
mean_val_loss = np.mean(history['val_loss'][10:20])
print(f"Mean val loss (epochs 11-20): {mean_val_loss:.2f}")

Mean val loss (epochs 11-20): 0.60


Answer: 0.88, the closest one

## Question 6

In [13]:
mean_val_acc_last5 = np.mean(history['val_acc'][15:20])
print(f"Mean val acc (last 5 epochs): {mean_val_acc_last5:.2f}")

Mean val acc (last 5 epochs): 0.69


Answer: 0.68, the closest one